## Deep Research

One of the classic cross-business Agentic use cases! This is huge.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Commercial implications</h2>
            <span style="color:#00bfff;">A Deep Research agent on Covered Strangles
            </span>
        </td>
    </tr>
</table>

In [1]:
# Imports

from agents import Agent, WebSearchTool, trace, Runner, function_tool
from agents.model_settings import ModelSettings
from pydantic import BaseModel
from dotenv import load_dotenv
import asyncio
import os
from IPython.display import display, Markdown
from pprint import pprint
import requests
load_dotenv(override=True)

# Constants

pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = "https://api.pushover.net/1/messages.json"

## OpenAI Hosted Tools

OpenAI Agents SDK includes the following hosted tools:

The `WebSearchTool` lets an agent search the web.  
The `FileSearchTool` allows retrieving information from your OpenAI Vector Stores.  
The `ComputerTool` allows automating computer use tasks like taking screenshots and clicking.

### Important note - API charge of WebSearchTool

This is costing me 2.5 cents per call for OpenAI WebSearchTool. That can add up to $2-$3 for the next 2 labs. We'll use low cost Search tools with other platforms, so feel free to skip running this if the cost is a concern.

Costs are here: https://platform.openai.com/docs/pricing#web-search

## We will be making 4 Agents:

1. Search Agent - searches online given a search term using an OpenAI hosted tool
2. Planner Agent - given a query from the user, come up with searches
3. Report Agent - make a report on results
4. Push Agent - send a notification to the user's phone with a summary

## Our First Agent: Search Agent

Given a Search term, search for it on the internet and summarize results.

In [37]:
date_end = "30 May 2025"

In [38]:
INSTRUCTIONS = f"You are a research assistant. For a Crypto Symbol Find Sentiment probabilities of upward momentum\
     or downward momentum by  {date_end}"

search_agent = Agent(
    name="Search agent",
    instructions=INSTRUCTIONS,
    tools=[WebSearchTool(search_context_size="low")],
    model="gpt-4.1-mini",
    model_settings=ModelSettings(tool_choice="required"),
)

In [ ]:
message = "Whats the forecast for ETH?"

with trace("Search"):
    result = await Runner.run(search_agent, message)

display(Markdown(result.final_output))

Take a look at the trace

https://platform.openai.com/traces

## Our Second Agent: Planner Agent

Given a query, come up with 5 ideas for web searches that could be run.

Use Structured Outputs as our way to ensure the Agent provides what we need.

In [4]:
# See note above about cost of WebSearchTool

HOW_MANY_SEARCHES = 5

INSTRUCTIONS = f"You are a helpful research assistant. Given a query, come up with a set of web searches \
to perform to best answer the query. Output {HOW_MANY_SEARCHES} terms to query for."

# We use Pydantic objects to describe the Schema of the output

class WebSearchItem(BaseModel):
    reason: str
    "Your reasoning for why this search is important to the query."

    query: str
    "The search term to use for the web search."


class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem]
    """A list of web searches to perform to best answer the query."""

# We pass in the Pydantic object to ensure the output follows the schema

planner_agent = Agent(
    name="PlannerAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4.1-mini",
    output_type=WebSearchPlan,
)

In [5]:

message = "What is the price prediction for ETH for May30th and June 31st of 2025?"

with trace("Search"):
    result = await Runner.run(planner_agent, message)
    pprint(result.final_output)

WebSearchPlan(searches=[WebSearchItem(reason='To find expert predictions and forecasts about the price of Ethereum (ETH) specifically for May 30th, 2025', query='Ethereum price prediction May 30 2025'), WebSearchItem(reason='To gather additional forecasts and analysis for Ethereum on the specific date June 31 2025, noting that this date is likely June 30 since June has 30 days', query='Ethereum price prediction June 30 2025'), WebSearchItem(reason='To understand general trends and expert opinions on Ethereum price outlook in mid-2025 as a broader context', query='Ethereum price forecast 2025'), WebSearchItem(reason='To find any recent news or developments that might impact Ethereum price leading up to mid-2025', query='Ethereum news 2024 2025 impact price'), WebSearchItem(reason='To check for reliable forecast platforms that provide future Ethereum price predictions with a timeline', query='Ethereum price prediction platforms 2025')])


## Our Third Agent: Writer Agent

Take the results of internet searches and make a report

In [6]:
INSTRUCTIONS = (
    "You are a senior researcher tasked with writing a cohesive report for a research query. "
    "You will be provided with the original query, and some initial research done by a research assistant.\n"
    "You should first come up with an outline for the report that describes the structure and "
    "flow of the report. Then, generate the report and return that as your final output.\n"
    "The final output should be in markdown format, and it should be lengthy and detailed. Aim "
    "for 5-10 pages of content, at least 1000 words."
)


class ReportData(BaseModel):
    short_summary: str
    """A short 2-3 sentence summary of the findings."""

    markdown_report: str
    """The final report"""

    follow_up_questions: list[str]
    """Suggested topics to research further"""


writer_agent = Agent(
    name="WriterAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=ReportData,
)

## Our Fourth Agent: push notification

Just to show how easy it is to make a tool!

I'm using a nifty product called PushOver - to set this up yourself, visit https://pushover.net

In [32]:
pushover_user = 'us34s3qa13fd7y8oomhq1mvib3qqy9'

pushover_token = 'ae1pdpx17zvyfvcgqrpf6i6zgh4wj4'

In [35]:
@function_tool
def push(message: str):
    """Send a push notification with this brief message"""
    payload = {"user": pushover_user, "token": pushover_token, "message": message}
    requests.post(pushover_url, data=payload)
    return {"status": "success"}

In [36]:
push

FunctionTool(name='push', description='Send a push notification with this brief message', params_json_schema={'properties': {'message': {'title': 'Message', 'type': 'string'}}, 'required': ['message'], 'title': 'push_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x121387560>, strict_json_schema=True)

In [9]:
INSTRUCTIONS = """You are a member of a research team and will be provided with a short summary of a report.
When you receive the report summary, you send a push notification to the user using your tool, informing them that research is complete,
and including the report summary you receive"""


push_agent = Agent(
    name="Push agent",
    instructions=INSTRUCTIONS,
    tools=[push],
    model="gpt-4.1-mini",
    model_settings=ModelSettings(tool_choice="required")
)

### The next 3 functions will plan and execute the search, using planner_agent and search_agent

In [11]:
async def plan_searches(query: str):
    """ Use the planner_agent to plan which searches to run for the query """
    print("Planning searches...")
    result = await Runner.run(planner_agent, f"Query: {query}")
    print(f"Will perform {len(result.final_output.searches)} searches")
    return result.final_output

async def perform_searches(search_plan: WebSearchPlan):
    """ Call search() for each item in the search plan """
    print("Searching...")
    tasks = [asyncio.create_task(search(item)) for item in search_plan.searches]
    results = await asyncio.gather(*tasks)
    print("Finished searching")
    return results

async def search(item: WebSearchItem):
    """ Use the search agent to run a web search for each item in the search plan """
    input = f"Search term: {item.query}\nReason for searching: {item.reason}"
    result = await Runner.run(search_agent, input)
    return result.final_output

### The next 2 functions write a report and send a push notification

In [12]:
async def write_report(query: str, search_results: list[str]):
    """ Use the writer agent to write a report based on the search results"""
    print("Thinking about report...")
    input = f"Original query: {query}\nSummarized search results: {search_results}"
    result = await Runner.run(writer_agent, input)
    print("Finished writing report")
    return result.final_output

async def send_push(report: ReportData):
    """ Use the push agent to send a notification to the user """
    print("Pushing...")
    result = await Runner.run(push_agent, report.short_summary)
    print("Push sent")
    return report

### Showtime!

In [13]:
query ="What is the prediction of ETH prices for May 2025"

with trace("Research trace"):
    print("Starting research...")
    search_plan = await plan_searches(query)
    search_results = await perform_searches(search_plan)
    report = await write_report(query, search_results)
    await send_push(report)  
    print("Hooray!")
display(Markdown(report.markdown_report))

Starting research...
Planning searches...
Will perform 5 searches
Searching...
Finished searching
Thinking about report...
Finished writing report
Pushing...
Push sent
Hooray!


# Prediction of Ethereum (ETH) Prices for May 2025

## Table of Contents

1. [Introduction](#introduction)
2. [Current Market Data](#current-market-data)
3. [Technical Analysis](#technical-analysis)
4. [Market Sentiment and External Factors](#market-sentiment-and-external-factors)
5. [Price Forecasts from Analysts](#price-forecasts-from-analysts)
6. [Factors Influencing Predictions](#factors-influencing-predictions)
7. [Conclusion](#conclusion)

## Introduction
Ethereum (ETH) has emerged as one of the leading cryptocurrencies due to its underlying technology, which powers a range of decentralized applications. As of May 2025, the quest for accurate predictions regarding Ethereum's price has gained momentum among investors, analysts, and traders. This report consolidates various data points and analyses to project ETH prices for May 2025, while highlighting the influencing factors that may impact its valuation.

## Current Market Data
As of May 17, 2025, Ethereum (ETH) is trading at **$2,504.98**, reflecting a slight decrease of 2.60% from the previous close. Below are the key trading statistics:
- **Current Price**: $2,504.98
- **Change**: -$66.73 (-0.03%) from previous close
- **Intraday High**: $2,631.38
- **Intraday Low**: $2,466.40  
  
This recent data showcases a volatile market, indicating significant fluctuations in investor sentiments.

## Technical Analysis
Technical analysis has become a critical methodology for predicting cryptocurrency prices. Notable patterns and indicators suggest potential price movements for Ethereum:
  
### Patterns
- **Ascending Triangle**: This pattern suggests potential bullish momentum, implying that ETH could break above notable resistance levels. 
- **Resistance & Support Levels**: Key resistance is observed at **$1,850**, with anticipated bullish targets ranging from **$1,950 to $2,129**. Conversely, support levels have been identified at **$1,752** and **$1,528**.  

### Fibonacci Retracement Levels
Currently, Ethereum is testing the **0.786 Fibonacci retracement level** around **$2,075**. A weekly close above **$2,100** may signal a shift towards the **0.618** Fibonacci level at **$2,508**. However, a drop below **$1,700** could indicate further weakness in the price trajectory.  

## Market Sentiment and External Factors
Market mood and sentiment play critical roles in determining cryptocurrency prices. Currently, several factors influence Ethereum's valuation:

### Approval of ETFs
Anticipated approval of Ethereum-based Exchange-Traded Funds (ETFs) by the U.S. Securities and Exchange Commission is expected to enhance market liquidity. While these ETFs will not accommodate staking, their approval could nurture investor confidence and improve ETH prices.  

### Institutional Interest
There has been increasing interest from institutional investors eager to explore the benefits of Ethereum and its capabilities in decentralized finance (DeFi) and non-fungible tokens (NFTs).  

## Price Forecasts from Analysts
Numerous analysts and financial institutions provide starkly different predictions for Ethereum's price by the end of 2025, with estimates reflecting the market's uncertainty:
- **Standard Chartered**: Revised its forecast for Ethereum to **$4,000**, down 60% due to increasing competition and declining revenue. 
- **Steno Research**: Predicts that ETH may outperform Bitcoin, with projections estimating a price increase to **$8,000**.
- **Changelly**: Offers a forecast range, estimating a price from **$5,907.41 to $7,194.28**, with an average of **$6,124.39**.
- **DigitalCoinPrice**: Projects a maximum of **$8,116.15** by December 2025, with an average price of **$7,258.18**.
- **PricePredictions.com**: Suggests a potential peak at **$7,436.90**, with an average price around **$6,828.86** in May 2025.  

These diverse forecasts highlight the uncertainty and complexities surrounding Ethereum's price dynamics.

## Factors Influencing Predictions
A multitude of factors could sway Ethereum’s market price in the upcoming months. Key influencing elements include:

1. **Technological Upgrades**: Continuous system improvements, notably the transition to Ethereum 2.0 and future upgrades, are designed to enhance scalability and performance, which could boost investor trust.
2. **Market Conditions**: Changes in regulation, broader market trends, and investor sentiment can significantly impact cryptocurrency valuations. Given that Ethereum supports numerous decentralized applications, robustness in these sectors reflects positively on ETH.
3. **Volatility**: The cryptocurrency market is notorious for its volatility. Price fluctuations are common, and any news events could drastically shift investor perspectives. 

## Conclusion
The price predictions for Ethereum (ETH) in May 2025 vary widely among analysts, reflecting the complexities of the cryptocurrency market. Current trading data indicates moderate stability at **$2,504.98**, while technical analyses suggest potential growth if resistance levels are breached. Nevertheless, upcoming technological advancements and institutional interests will likely play crucial roles in shaping market trends. Investors should remain cautious and perform thorough due diligence before making financial commitments in this dynamic and unpredictable environment.

### As always, take a look at the trace

https://platform.openai.com/traces